# Translate `dzn` to `smt2`

### Check Versions of Tools

In [2]:
import subprocess

output = subprocess.check_output("./optimathsat/bin/optimathsat -version", shell=True, universal_newlines=True)
output

'MathSAT5 version 1.6.4 (d39a648d6bb7) (Feb 25 2020 14:02:07, gmp 6.1.2, gcc 9.2.1, 64-bit)\n'

In [3]:
output = subprocess.check_output("./minizinc/bin/minizinc --version", shell=True, universal_newlines=True)
output

'MiniZinc to FlatZinc converter, version 2.4.3, build 122680178\nCopyright (C) 2014-2020 Monash University, NICTA, Data61\n'

In [4]:
output = subprocess.check_output("./z3/build/z3 --version", shell=True, universal_newlines=True)
output

'Z3 version 4.8.8 - 64 bit\n'

First generate the FlatZinc files using the MiniZinc tool. Make sure that a `smt2` folder is located inside `./minizinc/share/minizinc/`. Else, to enable OptiMathSAT's support for global constraints download the [smt2.tar.gz](http://optimathsat.disi.unitn.it/data/smt2.tar.gz) package and unpack it there using

```zsh
tar xf smt2.tar.gz $MINIZINC_PATH/share/minizinc/
```

If next output shows a list of `.mzn` files, then this dependency is satified.

In [5]:
output = subprocess.check_output("ls -la ./minizinc/share/minizinc/smt2/", shell=True, universal_newlines=True)
print(output)

total 292
drwxr-xr-x  2 kw kw 4096 Jan 15  2018 .
drwxr-xr-x 15 kw kw 4096 Apr  4 09:36 ..
-rw-r--r--  1 kw kw  328 Nov 13  2017 alldifferent_except_0.mzn
-rw-r--r--  1 kw kw  382 Nov 13  2017 all_different_int.mzn
-rw-r--r--  1 kw kw  396 Nov 13  2017 all_different_set.mzn
-rw-r--r--  1 kw kw  270 Nov 13  2017 all_disjoint.mzn
-rw-r--r--  1 kw kw  150 Nov 14  2017 all_equal_int.mzn
-rw-r--r--  1 kw kw  164 Nov 13  2017 all_equal_set.mzn
-rw-r--r--  1 kw kw  351 Nov 13  2017 among.mzn
-rw-r--r--  1 kw kw  305 Nov  8  2017 arg_max_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_max_int.mzn
-rw-r--r--  1 kw kw  306 Nov  8  2017 arg_min_float.mzn
-rw-r--r--  1 kw kw  291 Nov  8  2017 arg_min_int.mzn
-rw-r--r--  1 kw kw  480 Nov 13  2017 at_least_int.mzn
-rw-r--r--  1 kw kw  506 Nov 14  2017 at_least_set.mzn
-rw-r--r--  1 kw kw  340 Nov 13  2017 at_most1.mzn
-rw-r--r--  1 kw kw  474 Nov 13  2017 at_most_int.mzn
-rw-r--r--  1 kw kw  502 Nov 13  2017 at_most_set.mzn
-rw-r--r--  1 kw kw 1

## Transform `dzn` to `fzn` Using Model `mzn`

Then transform the desired `.dzn` file to `.fzn` using the `Mz.mzn` MiniZinc model.

In [6]:
import os

dzn_files = []
dzn_path = 'dzn-files/'

for filename in os.listdir(dzn_path):
    if filename.endswith(".dzn"):
        dzn_files.append(filename)
len(dzn_files)

21

In [7]:
fzn_path = 'fzn-files/'
minizinc_base_cmd = './minizinc/bin/minizinc \
    --compile --solver org.minizinc.mzn-fzn \
    --globals-dir smt2 dzn-files/Mz.mzn '
for dzn in dzn_files:
    minizinc_transform_cmd = minizinc_base_cmd + dzn_path + dzn \
        + ' --output-to-file ' + fzn_path + dzn.replace('.', '-') + '.fzn'
    print(f'''Translating {dzn_path + dzn} to {fzn_path + dzn.replace('.', '-')}.fzn''')
    subprocess.check_output(minizinc_transform_cmd, shell=True, 
                                     universal_newlines=True)

Translating dzn-files/R205.dzn to fzn-files/R205-dzn.fzn
Translating dzn-files/A004.dzn to fzn-files/A004-dzn.fzn
Translating dzn-files/A001.dzn to fzn-files/A001-dzn.fzn
Translating dzn-files/A010.dzn to fzn-files/A010-dzn.fzn
Translating dzn-files/A013.dzn to fzn-files/A013-dzn.fzn
Translating dzn-files/A006.dzn to fzn-files/A006-dzn.fzn
Translating dzn-files/A016.dzn to fzn-files/A016-dzn.fzn
Translating dzn-files/A007.dzn to fzn-files/A007-dzn.fzn
Translating dzn-files/A017.dzn to fzn-files/A017-dzn.fzn
Translating dzn-files/A008.dzn to fzn-files/A008-dzn.fzn
Translating dzn-files/A014.dzn to fzn-files/A014-dzn.fzn
Translating dzn-files/A018.dzn to fzn-files/A018-dzn.fzn
Translating dzn-files/A011.dzn to fzn-files/A011-dzn.fzn
Translating dzn-files/A009.dzn to fzn-files/A009-dzn.fzn
Translating dzn-files/A019.dzn to fzn-files/A019-dzn.fzn
Translating dzn-files/A005.dzn to fzn-files/A005-dzn.fzn
Translating dzn-files/A012.dzn to fzn-files/A012-dzn.fzn
Translating dzn-files/A015.dzn 

## Translate `fzn` to `smt2`

The generated `.fzn` file can be used to generate a `.smt2` file using the `fzn2smt2.py` script from this [project](https://github.com/PatrickTrentin88/fzn2omt).

In [8]:
import os

fzn_files = []
fzn_path = 'fzn-files/'

for filename in os.listdir(fzn_path):
    if filename.endswith(".fzn"):
        fzn_files.append(filename)
len(fzn_files)

21

In [10]:
smt2_path = 'smt2-files/'
fzn2smt2_base_cmd = './fzn2omt/fzn2smt2.py --solver z3 --smt2 '
for fzn in fzn_files:
    fzn2smt2_transform_cmd = fzn2smt2_base_cmd + smt2_path \
    + fzn.replace('.', '-') + '.smt2 ' + fzn_path + fzn
    print(f'''Translating {fzn_path + fzn} to {smt2_path + fzn.replace('.', '-')}.smt2''')
    my_env = os.environ.copy()
    my_env['PATH'] = "/home/kw/optimathsat/bin/:" + my_env['PATH']
    subprocess.check_output(fzn2smt2_transform_cmd, shell=True, env=my_env,
                                     universal_newlines=True)

Translating fzn-files/A009-dzn.fzn to smt2-files/A009-dzn-fzn.smt2
Translating fzn-files/A016-dzn.fzn to smt2-files/A016-dzn-fzn.smt2
Translating fzn-files/R205-dzn.fzn to smt2-files/R205-dzn-fzn.smt2
Translating fzn-files/A004-dzn.fzn to smt2-files/A004-dzn-fzn.smt2
Translating fzn-files/A013-dzn.fzn to smt2-files/A013-dzn-fzn.smt2
Translating fzn-files/A006-dzn.fzn to smt2-files/A006-dzn-fzn.smt2
Translating fzn-files/A011-dzn.fzn to smt2-files/A011-dzn-fzn.smt2
Translating fzn-files/A008-dzn.fzn to smt2-files/A008-dzn-fzn.smt2
Translating fzn-files/A014-dzn.fzn to smt2-files/A014-dzn-fzn.smt2
Translating fzn-files/A005-dzn.fzn to smt2-files/A005-dzn-fzn.smt2
Translating fzn-files/A002-dzn.fzn to smt2-files/A002-dzn-fzn.smt2
Translating fzn-files/A018-dzn.fzn to smt2-files/A018-dzn-fzn.smt2
Translating fzn-files/A017-dzn.fzn to smt2-files/A017-dzn-fzn.smt2
Translating fzn-files/A019-dzn.fzn to smt2-files/A019-dzn-fzn.smt2
Translating fzn-files/A001-dzn.fzn to smt2-files/A001-dzn-fzn.

This shoud generate the `smt2` file without any error. If this was the case then the `z3` prover can be called on a file by running


```zsh
z3 output/A001-dzn-smt2-fzn.smt2 
```

yielding something similar to

```zsh
z3 output/A001-dzn-smt2-fzn.smt2                                                       
sat
(objectives
 (obj 41881)
)
(model 
  (define-fun X_INTRODUCED_981_ () Bool
    false)
  (define-fun X_INTRODUCED_348_ () Bool
    false)
   
   .....
```

In [11]:
result = subprocess.check_output('./z3/build/z3 smt2-files/A012-dzn-fzn.smt2', 
                        shell=True, universal_newlines=True)
print(result)

sat
(objectives
 (obj 0)
)
(model 
  (define-fun X_INTRODUCED_19_ () Bool
    true)
  (define-fun X_INTRODUCED_5_ () Bool
    false)
  (define-fun X_INTRODUCED_8_ () Bool
    false)
  (define-fun X_INTRODUCED_3_ () Int
    1)
  (define-fun X_INTRODUCED_7_ () Bool
    false)
  (define-fun X_INTRODUCED_21_ () Bool
    false)
  (define-fun X_INTRODUCED_14_ () Int
    1)
  (define-fun X_INTRODUCED_9_ () Int
    0)
  (define-fun X_INTRODUCED_1_ () Int
    1)
  (define-fun obj () Int
    0)
  (define-fun X_INTRODUCED_0_ () Int
    2)
)

